In [ ]:

import warnings
import cv2
import time
import os
import sys
import pytesseract
import torch
import multiprocess
import numpy as np
from datasets import DatasetDict
from PIL import Image

# zeige keine Warnungen an
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.ocr_pipeline import OCRPreprocessor, OCRPostProcessor
from src.utils import rotate_image, pil_to_cv, from_cv_to_pil

torch.set_num_threads(1)

In [ ]:
# Datensatz initialisieren
dataset = DatasetDict.load_from_disk("../data/raw")

In [ ]:
from typing import Union
class OCRPipeline:
    def __init__(self, image: Union[np.ndarray, Image.Image]):
        """OCR-Pipeline zu Vorbereitung des Dokumentes, 
        Extraktion des Textes und Aufbereitung des extrahierten Textes.

        Args:
            Args:
            image (Union[np.ndarray, Image.Image]): Das Eingangsbild als NumPy-Array oder PIL.Image.Image.
        """
        self.raw_image = image
        self.preprocessed_image = None
        self.ocr_output = ""

    def preprocess(self) -> None:
        """Initialisiert und wendet den OCRPreprocessor an, speichert das verarbeitete Bild."""
        preprocessor = OCRPreprocessor(self.raw_image)
        preprocessor.cropping(buffer_size=10)
        preprocessor.resize(factor=3)
        preprocessor.contrast_stretching()
        preprocessor.power_law_transform(gamma=2)
        preprocessor.to_gray()
        #preprocessor.correct_skew()
        preprocessor.sharpen(kernel_type="laplace_standard")
        preprocessor.opening(kernel=(1,1), iterations=2)
        preprocessor.power_law_transform(gamma=2)
        self.preprocessed_image = preprocessor.get_image()

    def extract_text(self) -> None:
        """Wendet PyTesseract auf das vorverarbeitete Bild an und speichert den Text."""
        self.ocr_output = pytesseract.image_to_string(self.preprocessed_image)

    def postprocess(self) -> None:
        """Initialisiert und wendet den OCRPostProcessor auf den extrahierten Text an."""
        if self.ocr_output.strip():  # Prüft, ob `ocr_output` nicht leer ist
            postprocessor = OCRPostProcessor(self.ocr_output)
            # Anwenden verschiedener Methoden
            postprocessor.identify_language()
            postprocessor.remove_special_characters()
            postprocessor.remove_stopwords()
            #postprocessor.stem()
            #postprocessor.lemmatize()
            postprocessor.lowercase()
            postprocessor.spellcheck(checker_type="symspell")
            
            # Aufbereiteten OCR-Output extrahieren
            self.ocr_output = postprocessor.get_text()
        else:
            self.ocr_output = "no text found in image with ocr!"

    def get_output(self):
        """Gibt den aufbereiteten OCR-Output zurück."""
        return self.ocr_output

In [ ]:
def apply_ocr_to_dataset(dataset: DatasetDict) -> DatasetDict:
    """
    Diese Methode wendet die OCR (Optical Character Recognition) auf alle Bilder in jedem Split (train, validation, test) eines Huggingface-Datensatzes an und fügt ein neues Feature hinzu, das den erkannten Text enthält.
    """
    def add_ocr_text(example: dict) -> dict:
        image = example['image']
            
        ocr_pipeline = OCRPipeline(image)
            
        ocr_pipeline.preprocess()

        ocr_pipeline.extract_text()
            
        ocr_pipeline.postprocess()

        example['tesseract_text'] = ocr_pipeline.get_output()
            
        return example
    
     # Anwenden der Funktion auf jeden Split im Datensatz
    for split in dataset.keys():
        dataset[split] = dataset[split].map(add_ocr_text, keep_in_memory=False)
            
    return dataset

In [ ]:
%%time
processed_dataset = apply_ocr_to_dataset(dataset)

In [ ]:
%%time
processed_dataset = apply_ocr_to_dataset(dataset)

In [ ]:
processed_dataset["test"][0]

### Prüfen ob kein string im Feature "Text" leer ist in allen drei Datensätzen

In [ ]:
processed_dataset.save_to_disk("../data/processed")